In [1]:
import pickle
import os

###Mount data

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g7bZe7HvJncZjjRGrIR8lN_qcDii86Vk5TK7T_6sICT3gg5GqulvOc


KeyboardInterrupt: ignored

In [ ]:
#!ls drive/'My Drive'

In [ ]:
drive_path = 'drive/My Drive/Project22'

In [ ]:
print(os.getcwd())
#if not os.getcwd() == '/content/drive/My Drive/Project22':
os.chdir(drive_path)
#  os.getcwd()

In [ ]:
pip install mido

In [ ]:
# GLOBALS

SEQ_LENGTH = 256
TARGET_LENGTH = 1
NUM_CLASSES = 135




### Dataloader


In [ ]:
import numpy as np
import pandas as pd
import os
from math import floor
from midi_conversion import convert_midi_to_numpy, convert_matrix_to_word_seq, convert_to_number_seq
from torch.utils import data
import random
from time import time

#from PIL import Image, ImageOps
#from skimage.io import imread
#from skimage.transform import resize
#from sklearn.model_selection import StratifiedShuffleSplit
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
    """class Dataset(data.Dataset): 
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        # Return the size of the dataset
        return len(self.targets)

    def __getitem__(self, index):
        # Retrieve inputs and targets at the given index
        X = self.inputs[index]
        y = self.targets[index]
        return X, y
"""

In [ ]:
class load_data():
    # data_train, data_test and le are public
    def __init__(self, print_status = False, ticks_per_beat = 12):
        track_paths = self._generate_paths()
        self.ticks_per_beat = ticks_per_beat
        self.print_status = print_status
        self._load(track_paths)

    def _generate_paths(self):
        #allowed_folders = ["2004","2006","2008","2009","2011","2013","2014","2015","2017","2018"]
        allowed_folders = ["2004","2006","2008","2009"]
        paths = []
        maestro_dir = "./maestro-v2.0.0/"
        for maestro_folder in os.listdir(maestro_dir):
            if maestro_folder in allowed_folders:
                for track_path in os.listdir(maestro_dir + maestro_folder):
                    paths.append(maestro_dir + maestro_folder +
                                 "/" + track_path)
                  
        return paths

    def _load(self, track_paths):
        test_fraction = 0.1
        self.train_data = []
        self.test_data = []
        # debug
        t0 = time()
        load_times = []
        tracks_done = 0

        # the real dataloader
        for path in track_paths:
            t1 = time() #debug
            track = convert_midi_to_numpy(path, ticks_per_beat = self.ticks_per_beat) #convert to vector representation
            track = convert_matrix_to_word_seq(track) #convert to word seq
            track = convert_to_number_seq(track) #convert to number representation
            track_np = np.array(track) #convert to numpy
            if random.random() > test_fraction: self.train_data.append(track_np) # add to train
            else: self.test_data.append(track_np) # add to test
            #debug
            tracks_done += 1
            t2 = time()
            load_time = t2-t1
            load_times.append(load_time)
            tracks_left = len(track_paths) - tracks_done
            load_time_avr = sum(load_times)/len(load_times)
            if self.print_status: print(path[-20:],"took",str(load_time)[:4],"s,",tracks_done,"out of",len(track_paths),": time left", str((tracks_left*load_time_avr)/60)[:4],"minutes")
        
        """
        #test dataloader
        for i in range(250):
            track_length = 1000
            note = random.randint(0,128)
            track = np.zeros((track_length,129))
            for e in range(track_length): #line
                track[e][note] = 1
            train_data.append(track)
            test_data.append(track)
        """

        print("train:",len(self.train_data))
        print("test:",len(self.test_data))

        #cut to 1024 pieces (snack pieces)
        #self.train_cut = self._chop_data(self.train_uncut)
        #self.test = self._chop_data(self.test_uncut)
        #self.train_targets = self._chop_data(self.train_uncut_shifted)
        #self.test_targets = self._chop_data(self.test_uncut_shifted)
        #self.train = Dataset(self.train_cut, self.train_targets)

    def generate_cut_data(self, cut_length = 128):
        self.cut_length = cut_length
        #chop and generate targets
        train_data_cut, train_data_cut_targets = self._chop_data(self.train_data)
        test_data_cut, test_data_cut_targets = self._chop_data(self.test_data)

        #convert to numpy, rename data and add extra pseduo dimension
        self.train_cut = np.expand_dims(np.array(train_data_cut),2)
        self.train_cut_targets = np.array(train_data_cut_targets)
        self.test_cut = np.expand_dims(np.array(test_data_cut),2)
        self.test_cut_targets = np.array(test_data_cut_targets)

    def _shift_data(self,data, n=1):
        data = np.roll(data, -n, axis = 0)
        data[-1] = 0 #makes the very last number 0 to keep the correct size
        return data

    def _chop_data(self, uncut_data):
        seq_length = self.cut_length
        all_tracks_cut = []
        all_tracks_cut_targets = []
        for track in uncut_data:
            track_pieces_length = floor(len(track)/seq_length)
            leftover_length = len(track)%seq_length
            #cut track
            for i in range(track_pieces_length):
                if (i+1)*seq_length+TARGET_LENGTH > len(track): break #if its too short for creating target, stop
                all_tracks_cut.append(track[i*seq_length:(i+1)*seq_length])
                all_tracks_cut_targets.append(track[(i+1)*seq_length : (i+1)*seq_length+TARGET_LENGTH]) #first part of next cut
            #and the tail. adds trailing zeroes to make up for missing length 
            #ACUTALLY WHATEVER THE TAIL ITS JUST ZEROES EITHER WAY ---
            #leftover = track[seq_length*track_pieces_length:] + [0 for _ in range(track_pieces_length - leftover_length)]
            #cut_track.append(leftover)

        return all_tracks_cut, all_tracks_cut_targets
            


In [ ]:
class batch_generator():
    def __init__(self, data, batch_size=64,
                 num_iterations=5e3, num_features=1, seed=42, val_size=0.1):
        self._train = data.train_cut
        self._train_targets = data.train_cut_targets
        self._test = data.test_cut #TODO WE DON'T USE THIS YET
        self._test_targets = data.test_cut_targets #TODO WE DON'T USE THIS YET

        self._batch_size = batch_size
        self._seq_length = SEQ_LENGTH
        self._num_iterations = num_iterations
        self._num_features = num_features
        self._seed = seed
        self._val_size = val_size
        self._valid_split()

    def _valid_split(self):
        p = self._val_size
        val_filter = [True if p < random.random() else False for _ in range(len(self._train))] #some filter to choose valid from
        train_filter = [not val for val in val_filter]
        #while True: a = 0 #nice debug break
        self._idcs_valid = np.arange(len(self._train))[val_filter]
        self._idcs_train = np.arange(len(self._train))[train_filter]

    def _shuffle_train(self):
        np.random.shuffle(self._idcs_train)

    def _batch_init(self, purpose):
        assert purpose in ['train', 'valid', 'test']

        batch_holder = dict()

        batch_holder['inputs'] = np.zeros(
            (self._batch_size, self._seq_length, 1), dtype="float32")
        batch_holder['targets'] = np.zeros(
            (self._batch_size, TARGET_LENGTH), dtype="float32")
        return batch_holder

    def gen_valid(self):
        batch = self._batch_init(purpose='valid')
        i = 0
        for idx in self._idcs_valid:
            batch['inputs'][i] = self._train[idx]
            batch['targets'][i] = self._train_targets[idx]
            i += 1
            if i >= self._batch_size:
                yield batch, i
                batch = self._batch_init(purpose='valid')
                i = 0
        #if i != 0:
        #    batch = batch[:i]
        #    yield batch, i

    def gen_test(self): #TODO THIS IS PROBABLY BROKEN FOR NOW
        batch = self._batch_init(purpose='test')
        i = 0
        for idx in range(len(self._test)):
            batch[i] = self._test[idx]
            i += 1
            if i >= self._batch_size:
                yield batch, i
                batch = self._batch_init(purpose='test')
                i = 0
        if i != 0:
            yield batch, i

    def gen_train(self):
        batch = self._batch_init(purpose='train')
        iteration = 0
        i = 0
        while True:
            # shuffling all batches between batches. Shuffles idcs so it keeps the targets
            self._shuffle_train()
            for idx in self._idcs_train:
                # extract data from dict
                batch['inputs'][i] = self._train[idx]
                batch['targets'][i] = self._train_targets[idx]
                i += 1
                if i >= self._batch_size:
                    yield batch
                    batch = self._batch_init(purpose='train')
                    i = 0
                    iteration += 1
                    if iteration >= self._num_iterations:
                        break

### Load Data

In [ ]:
data = load_data(print_status = True) # UNCOMMENT FOR DATA LOADER

In [ ]:
#data.train_data[0][:10] #DEBUG

In [ ]:
# CUT DATA
data.generate_cut_data(SEQ_LENGTH)

In [ ]:
#i = 4124
#print(data.train_cut[i])
#print(data.train_cut_targets[i])

In [ ]:
# DATA DEBUG
#data.train_data = [list(range(457)) for _ in range(10000)]

In [ ]:
# PICKLE STUFF
# Dump the data into a pickle file
"""
with open('data.pickle_half_tracks_cut', 'wb') as f:
  pickle.dump(data, f)

"""

with open('data.pickle_half_tracks_cut', 'rb') as f:
    data = pickle.load(f)



In [ ]:
#Some debugging stuff
#data.train_cut[0]

In [ ]:
"""for o in range(257,300):
    n = np.count_nonzero(data.train_cut == o)
    s = data.train_cut.size
    print((n/s)*100,"% is ",o," i.e. wait")"""

In [ ]:
np.count_nonzero(data.train_cut > 120)/data.train_cut.size*100 # % percents are wait

In [ ]:
#data.train_data = old_train_data

In [ ]:
# DISTRUBUTE WAITS OVER RANGE
"""for i, track in enumerate(data.train_data):
    c = 0
    for e, num in enumerate(track):
        if num > 258: # is wait
            data.train_data[i][e] = random.randint(259,300)
"""

In [ ]:

np.unique(data.train_cut)

data.train_cut.shape


In [ ]:
np.max(data.train_cut)

In [ ]:
data_distr = np.bincount(data.train_cut.reshape((-1)))
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = list(range(len(data_distr)))
y = data_distr
ax.bar(x,y)
axes = plt.gca()
#axes.set_ylim([0,100000])
#axes.set_xlim([0,120])
plt.show()

In [ ]:
# and its corresponding target
#plot_midi(data.train_cut_targets[10][:],seq_length = TARGET_LENGTH)
#%debug

In [ ]:
#mone = np.full((151337, 1), 1)
#data.train_cut = np.subtract(data.train_cut, mone)

In [ ]:
#np.unique(data.train_cut_targets)

### Network

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d, BatchNorm2d, ReLU
from torch.nn.functional import relu, elu, relu6, tanh, softmax

lstm_hidden = 50
seq_length = SEQ_LENGTH
OUT_CLASSES = 135
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()


        self.lstm_1 = nn.LSTM(input_size=1,
                              hidden_size=lstm_hidden,
                              num_layers=1,
                              bidirectional=False,
                              batch_first = True)


        self.l_hidden = Linear(in_features = lstm_hidden*seq_length,
                                    out_features = OUT_CLASSES,
                                    bias=False)
        """

        self.l_hidden = Linear(in_features = seq_length,
                                    out_features = SEQ_LENGTH*50,
                                    bias=False)
    
        self.l_hidden2 = Linear(in_features = SEQ_LENGTH*50,
                                    out_features = NUM_CLASSES,
                                    bias=False)
        """
        

        self.dropout = Dropout(0.1)
        


    def forward(self, inp):
        
        #features = []
        out = {}
        

        #x, (h, c) = self.lstm_1(x)
        #x,_ = self.lstm_1(inp)

        #print("x",x.shape)
        #print("h",h)
        #print("c",c)
        #print("----")

        #features.append(x)
        # Final concatenated data processing
        #features_final = torch.cat(features, dim=1)


        #print("x0",x)

        #x = torch.reshape(x,(-1,3))
        #x = torch.transpose(x,1,0)

        x, _ = self.lstm_1(inp)
        #x = inp
        x = torch.reshape(x,(x.shape[0],-1))
        x = self.dropout(x)
        x = torch.sigmoid(self.l_hidden(x))
        #x = torch.sigmoid(self.l_hidden2(x))
        out['out'] = np.squeeze(x)

        return out

use_cuda = torch.cuda.is_available() #TODO ALLOW AGAIN
net = Net()
if use_cuda:
    net.cuda()

print(net)


In [ ]:
# a small sanity check
#%debug

inp = torch.randn(500, SEQ_LENGTH, 1).cuda()

outp,hidden = net(inp)
outp,hidden = net(inp)

for key in outp:
    print(key,outp[key].shape)
    #print(outp[key][0][1])


### Training loop

In [ ]:
from numpy.lib.npyio import load
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d, BatchNorm2d, ReLU
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
import matplotlib.pyplot as plt
from IPython.display import clear_output
from math import log

In [ ]:
VALIDATION_SIZE = 0.1  # 0.1 is ~ 100 samples for validation
max_iter = 2000
log_every = 100
eval_every = 100
batch_size = 500

net = Net()
if use_cuda:
    net.cuda()

def simple_crit(inputs,targets):
    b = torch.abs(torch.subtract(inputs,targets)-0.001)
    a = -torch.log(b)
    return torch.mean(a)

LEARNING_RATE = 0.0001
#criterion = nn.BCEWithLogitsLoss(reduction = "mean")

#normie weights
#loss_weights = torch.zeros(135)
#loss_weights[0:62*2] = 1
#loss_weights[62*2:] = 0.01
#loss_weights = loss_weights.cuda()

#distr weights
data_distr = np.bincount(data.train_cut.reshape((-1)))
ddd = torch.FloatTensor([1/(data_distr[i]*0.001) for i in range(135)]).cuda()
criterion = nn.CrossEntropyLoss(ddd)
#criterion = simple_criterion
#criterion = simple_crit
#criterion = nn.MSELoss()

def accuracy(ys, ts):
    ys = torch.argmax(ys, dim=1)
    correct_prediction = torch.eq(ys, ts)
    return torch.mean(correct_prediction.float())

# Function to get label
def get_labels(batch):
    return get_variable(Variable(torch.from_numpy(batch['ts']).long()))

# weight_decay is equal to L2 regularization
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)#, weight_decay=1e-5)

# Initialize lists for training and validation
train_iter = []
train_loss, train_accs = [], []
valid_iter = []
valid_loss, valid_accs = [], []

# Generate batches
batch_gen = batch_generator(data,
                                batch_size=batch_size,
                                num_iterations=max_iter,
                                seed=42,
                                val_size=VALIDATION_SIZE)


def get_input(batch):
    return get_variable(Variable(torch.from_numpy(batch)))

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()


# Train network
net.train()
train_idx = 0

h = torch.randn(1, 500, lstm_hidden).cuda()
c = torch.randn(1, 500, lstm_hidden).cuda()
hidden = (h,c)

for i, batch_train in enumerate(batch_gen.gen_train()):

    if i % eval_every == 0:
        # Do the validaiton
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0

        #batches = batch_gen.gen_valid()
        for batch_valid, num in batch_gen.gen_valid():
            output,_ = net(get_input(batch_valid['inputs']))
            #print("hej")
            #print(output['out'])
            #print(criterion(output['out'], get_input(batch_valid['targets'])))
            val_losses += criterion(output['out'], get_input(np.squeeze(batch_valid['targets'])).long()) * num
            val_accs += accuracy(output['out'], get_input(np.squeeze(batch_valid['targets']))) * num
            val_lengths += num

        # Divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        valid_loss.append(get_numpy(val_losses))
        valid_accs.append(get_numpy(val_accs))
        valid_iter.append(i)
   
        net.train()

    # Train network
    output,_ = net(get_input(batch_train['inputs']))

    #labels_argmax = torch.max(get_labels(batch_train), 1)[1]
    #validation_batch = get_variable(torch.transpose(torch.from_numpy(batch_train).long(), 0, 1))
    #validation_batch = get_variable(torch.transpose(torch.from_numpy(batch_train).long(), 0, 1))
    batch_loss = criterion(output['out'], get_input(np.squeeze(batch_train['targets'])).long())
    train_iter.append(i)
    train_loss.append(float(get_numpy(batch_loss)))
    train_accs.append(float(get_numpy(accuracy(output['out'], get_input(batch_train['targets'])))))

    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    # Log i figure
    if i % log_every == 0:
        fig = plt.figure(figsize=(18, 6))
        plt.subplot(1, 2, 1)
        plt.plot(train_iter, train_loss, label='train_loss')
        plt.plot(valid_iter, valid_loss, label='valid_loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(train_iter, train_accs, label='train_accs')
        plt.plot(valid_iter, valid_accs,
                  label='valid_accs: ' + str(valid_accs[-1]))
        plt.legend()
        plt.show()
        clear_output(wait=True)
   
    if max_iter < i:
        break


#val = torch.max([])


In [ ]:
%debug

In [ ]:
i = 2
print(np.squeeze(batch_train["inputs"][i]))
np.squeeze(batch_train["targets"][i])

In [ ]:
"""
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
#output.backward()
loss = nn.CrossEntropyLoss()

input = torch.from_numpy(np.array([[1.0,1.0,0.0]]))
target = torch.from_numpy(np.array([1]))
loss(input, target)
"""

In [ ]:
#notes_np = midi_conversion.convert_midi_to_numpy("maestro-v2.0.0/2008/MIDI-Unprocessed_04_R1_2008_01-04_ORIG_MID--AUDIO_04_R1_2008_wav--4.midi")

In [ ]:
from midi_plotter import plot_midi, plot_num_seq

In [ ]:
"""i = 4
inp = batch_train['inputs'][i]
tar = batch_train['targets'][i]"""

In [ ]:
#plot_midi(inp,show_plot=True, seq_length=25)

In [ ]:
#plot_midi(tar,show_plot=True, seq_length=TARGET_LENGTH)

In [ ]:
import midi_conversion

In [ ]:
r = random.randint(0,99)
print(r)
#inp_np = np.array(batch_train['inputs'])[r] # +np.full((128,1),5) #only one batch
inp_np = batch_valid['inputs'][r] # +np.full((128,1),5) #only one batch
print(np.squeeze(inp_np))
net.eval()
all_removed = []
for i in range(1000):
    #inp = torch.from_numpy(np.array([inp_np])).float().cuda()
    inp = get_input(np.array([inp_np]))
    outp = net(inp)['out']
    outp = outp.cpu().detach()
    pred = torch.argmax(outp).numpy()
    #print("pred",pred)
    all_removed.append(inp_np[0])
    inp_np = np.roll(inp_np,-1,0)
    inp_np[-1] = [pred]

np.squeeze(inp_np)

In [ ]:
num_seq = [int(num) for num in np.squeeze(inp_np)]
all_removed = [int(num) for num in all_removed]
num_seq = all_removed+num_seq
midi_conversion.convert_num_seq_to_midi(num_seq,output_name="GENERATED")

In [ ]:
%debug

In [ ]:
###### OUTPUT NETWORK TEST ###########
#get an input and convert
i = 4
inp = batch_train['inputs']
tar = batch_train['targets']
inp = torch.from_numpy(np.array(inp)).float().cuda()
#inp = torch.from_numpy(np.array(inp)).float()
inp.shape

In [ ]:
#get networks output
net.eval()
outp = net(inp)['out']
outp = outp.cpu().detach()
#outp = torch.argmax(outp,dim=1)

In [ ]:
tar = torch.from_numpy(tar)

In [ ]:
torch.mean(torch.eq(outp,tar).float())

In [ ]:
torch.argmax(outp,1)

In [ ]:
#analyse
print("Shape of outp:",outp.shape)

In [ ]:
tar

In [ ]:
outp[259]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = list(range(458))
y = outp
ax.bar(x,y)
plt.show()

In [ ]:
outp = np.where(outp > 0.9, 1, 0)
plot_num_seq(outp,show_plot=True, seq_length=TARGET_LENGTH)

In [ ]:
#plotting target again
plot_midi(tar,show_plot=True, seq_length=TARGET_LENGTH)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(de, bins = 10)